In [11]:
# For Preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score,recall_score,precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import RandomOverSampler

In [12]:
X = pd.read_csv("application_train.csv")
#df_test = pd.read_csv("application_test.csv")
X.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
X['CODE_GENDER'].replace('XNA',np.nan, inplace=True)
X['CODE_GENDER'].value_counts()

F    202448
M    105059
Name: CODE_GENDER, dtype: int64

In [14]:
X['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [15]:
X['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [16]:
X.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in X:
    if X[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(X[col].unique())) <= 2:
            # Train on the training data
            le.fit(X[col])
            # Transform both training and testing data
            X[col] = le.transform(X[col])
           # df_test[col] = le.transform(df_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)


3 columns were label encoded.


In [18]:
# one-hot encoding of categorical variables
X = pd.get_dummies(X)
#df_test = pd.get_dummies(df_test)
list1=X.columns

print('Training Features shape: ', X.shape)
#print('Testing Features shape: ', df_test.shape)
print(list1)

Training Features shape:  (307511, 242)
Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'HOUSETYPE_MODE_terraced house', 'WALLSMATERIAL_MODE_Block',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'EMERGENCYSTATE_MODE_No', 'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=242)


In [19]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [20]:
# Missing values statistics
missing_values = missing_values_table(X)
mis_col=list(missing_values.index)
Mis=X[mis_col][:]

for col in mis_col:
    X[col+'_miss']=[0]*len(X[col])
    X[col+'_miss'][list(X[np.isnan(X[col])].index)]=1

print(X.describe)

Your selected dataframe has 242 columns.
There are 62 columns that have missing values.


C:\Users\rajes\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<bound method NDFrame.describe of         SK_ID_CURR  TARGET  NAME_CONTRACT_TYPE  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0           100002       1                   0             0                1   
1           100003       0                   0             0                0   
2           100004       0                   1             1                1   
3           100006       0                   0             0                1   
4           100007       0                   0             0                1   
5           100008       0                   0             0                1   
6           100009       0                   0             1                1   
7           100010       0                   0             1                1   
8           100011       0                   0             0                1   
9           100012       0                   1             0                1   
10          100014       0                   0             0               

[307511 rows x 304 columns]>


In [21]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
print('before...')    
Mis=X[mis_col][:]
print('Mis - ',Mis)

xt = DataFrameImputer().fit_transform(Mis)

print('after...')
print(xt)

before...
Mis -          COMMONAREA_MODE  COMMONAREA_AVG  COMMONAREA_MEDI  \
0                0.0144          0.0143           0.0144   
1                0.0497          0.0605           0.0608   
2                   NaN             NaN              NaN   
3                   NaN             NaN              NaN   
4                   NaN             NaN              NaN   
5                   NaN             NaN              NaN   
6                   NaN             NaN              NaN   
7                   NaN             NaN              NaN   
8                   NaN             NaN              NaN   
9                   NaN             NaN              NaN   
10                  NaN             NaN              NaN   
11                  NaN             NaN              NaN   
12                  NaN             NaN              NaN   
13               0.0587          0.0582           0.0585   
14               0.1153          0.1143           0.1150   
15                  NaN

[307511 rows x 62 columns]
after...
        COMMONAREA_MODE  COMMONAREA_AVG  COMMONAREA_MEDI  \
0              0.014400        0.014300         0.014400   
1              0.049700        0.060500         0.060800   
2              0.042553        0.044621         0.044595   
3              0.042553        0.044621         0.044595   
4              0.042553        0.044621         0.044595   
5              0.042553        0.044621         0.044595   
6              0.042553        0.044621         0.044595   
7              0.042553        0.044621         0.044595   
8              0.042553        0.044621         0.044595   
9              0.042553        0.044621         0.044595   
10             0.042553        0.044621         0.044595   
11             0.042553        0.044621         0.044595   
12             0.042553        0.044621         0.044595   
13             0.058700        0.058200         0.058500   
14             0.115300        0.114300         0.115000   
15  

[307511 rows x 62 columns]


In [22]:
# #Have commented the dropping (not dropping any columns for now) Replace missing with -999

# X_mis=(Mis.isnull().sum() / Mis.shape[0] * 100.00)
# df_mis=pd.DataFrame(np.c_[X_mis.index,X_mis],columns=['Colname','miss_p'])
# drop_col=df_mis[df_mis['miss_p']>65]['Colname'].values
# print('number of columns before dropping->',len(X.columns))
# #X=X.drop(drop_col,axis=1)
# print('number of columns after dropping->',len(X.columns))
# X.replace(np.nan,-999, inplace=True)
# #X.astype('float32',inplace=True)
#print(newX[].describe())
for mis in mis_col:
    X[mis]=xt[mis]
print(X.shape)
print(X.describe)

(307511, 304)
<bound method NDFrame.describe of         SK_ID_CURR  TARGET  NAME_CONTRACT_TYPE  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0           100002       1                   0             0                1   
1           100003       0                   0             0                0   
2           100004       0                   1             1                1   
3           100006       0                   0             0                1   
4           100007       0                   0             0                1   
5           100008       0                   0             0                1   
6           100009       0                   0             1                1   
7           100010       0                   0             1                1   
8           100011       0                   0             0                1   
9           100012       0                   1             0                1   
10          100014       0                   0             0 

[307511 rows x 304 columns]>


In [23]:
# iterate below functions only

In [24]:
y=X['TARGET']
X=X.drop('TARGET',axis=1)

def split_train_test(X , y):
    return(train_test_split( X , y , test_size=0.2, random_state=42))

X_train, X_test, y_train, y_test =split_train_test(X,y)

In [25]:
def feature_eng(X_train,y_train,X_test,y_test):
    train_col=list(X_train.columns)
    test_col=list(X_test.columns)    
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X_train, y_train)
    model = SelectFromModel(clf, prefit=True)
    new_trainX = model.transform(X_train)
    n=np.shape(new_trainX)[1]
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    clist=[]
    for f in range(n):     
        clist.append(indices[f])
    new_train_col=[train_col[i] for i in clist]
    new_test_col=[test_col[i] for i in clist]
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    y_train=pd.DataFrame(y_train)
    y_test=pd.DataFrame(y_test)
    df_new_train=pd.DataFrame(X_train.loc[:,new_train_col],columns=new_train_col)            
    df_new_test=pd.DataFrame(X_test.loc[:,new_test_col],columns=new_test_col)
    print("Improved number of features->",len(clist))    
    return(df_new_train,df_new_test)



In [26]:
def run_grid_knn(X_train,Y_train,X_test,Y_test):
    # K Nearest Neighbor
    param_knn={'algorithm':['auto'],              
              "n_neighbors" : [i for i in range(1, 10)]}                      
    knearest = KNeighborsClassifier(n_jobs=-1)
    modelobj =  RandomizedSearchCV(estimator =knearest, param_distributions=param_knn, n_iter = 7, cv=3, scoring='f1',verbose=2,random_state=42, n_jobs = -1)    
    modelobj.fit(X_train,Y_train)
    eval_metrics('K NEAREST Neighbor:',modelobj,X_train,Y_train,X_test,Y_test) 

def eval_metrics(modelname,modelobj,X_train,Y_train,X_test,Y_test):
    write_file('CV Results-->',modelobj.cv_results_)
    print('Best parameter on grid-->',modelobj.best_params_)
    ypred_train=modelobj.predict(X_train)
    ypred_test=modelobj.predict(X_test)
    y_train_proba = modelobj.predict_proba(X_train)[:,1]
    y_test_proba = modelobj.predict_proba(X_test)[:,1]
    print(modelname,"model:")
    confmat=confusion_matrix(Y_train,ypred_train)
    print("Train Confusion Matrix:")
    print(confmat)
    confmat=confusion_matrix(Y_test,ypred_test)
    print("Test Confusion Matrix:")
    print(confmat)
    metric=pd.DataFrame([[0]*5]*2,columns=['PRECISION','RECALL','F1_SCORE','ROC_AUC_SCORE','ACCURACY'],index=['Train','Test'],dtype='float32')
    metric['PRECISION']['Train']=precision_score(Y_train, ypred_train)
    metric['RECALL']['Train']=recall_score(Y_train, ypred_train)
    metric['F1_SCORE']['Train']=f1_score(Y_train, ypred_train)
    metric['ROC_AUC_SCORE']['Train']=roc_auc_score(Y_train,y_train_proba)
    metric['ACCURACY']['Train']=accuracy_score(Y_train,ypred_train)
    metric['PRECISION']['Test']=precision_score(Y_test, ypred_test)
    metric['RECALL']['Test']=recall_score(Y_test, ypred_test)
    metric['F1_SCORE']['Test']=f1_score(Y_test, ypred_test)
    metric['ROC_AUC_SCORE']['Test']=roc_auc_score(Y_test,y_test_proba)
    metric['ACCURACY']['Test']=accuracy_score(Y_test,ypred_test)
    print(metric)
   

In [27]:
text_file = open("cvres_knn_imb_grid.txt", "w")
text_file.write('K NEAREST Neighbor SMOTE/OVER/UNDER SAMPLING GRID results:')
text_file.close()
def write_file(par,val):
    str_p='\n'+par+str(val)
    text_file = open("cvres_knn_imb_grid.txt", "a+")
    text_file.write(str_p)

In [28]:
#Handling Imbalance problem
X_tr, X_test, y_tr, y_test =split_train_test(X,y)
'''
Use SMOTE to handle imbalance problem
'''
text_file = open("cvres_knn_imb_grid.txt", "a+")
text_file.write('\nSMOTE:')
text_file.close()
# Class0 size is 10 times that of Class1 in train set
print('Y train before resampling',Counter(y_tr))
for i in range(15,100,15):
    print('Sampling strategy-->',i*0.01)
    write_file('Sampling strategy-->',i*0.01)  
    Imb_sm = SMOTE(random_state=42,sampling_strategy=i*0.01,n_jobs=-1)
    X_train, y_train = Imb_sm.fit_resample(X_tr, y_tr)
    print('Y train after resampling',Counter(y_train))
    write_file('Y train after resampling-->',Counter(y_train))        
    X_train=pd.DataFrame(X_train)
    y_train=pd.DataFrame(y_train)
    df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)
    run_grid_knn(df_new_train,y_train,df_new_test,y_test)

Y train before resampling Counter({0: 226132, 1: 19876})
Sampling strategy--> 0.15
Y train after resampling Counter({0: 226132, 1: 33919})


C:\Users\rajes\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Improved number of features-> 104
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:  3.1min remaining:   43.2s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  3.3min finished
C:\Users\rajes\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best parameter on grid--> {'n_neighbors': 1, 'algorithm': 'auto'}
K NEAREST Neighbor: model:
Train Confusion Matrix:
[[226132      0]
 [     0  33919]]
Test Confusion Matrix:
[[51118  5436]
 [ 4324   625]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000       1.000000  1.000000
Test    0.103118  0.126288  0.113533       0.515084  0.841309
Sampling strategy--> 0.3
Y train after resampling Counter({0: 226132, 1: 67839})


...

Best parameter on grid--> {'n_neighbors': 1, 'algorithm': 'auto'}
K NEAREST Neighbor: model:
Train Confusion Matrix:
[[226132      0]
 [     0 169599]]
Test Confusion Matrix:
[[47376  9178]
 [ 3931  1018]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000       1.000000  1.000000
Test    0.099843  0.205698  0.134434       0.521705  0.786856
Sampling strategy--> 0.9
Y train after resampling Counter({0: 226132, 1: 203518})


C:\Users\rajes\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Improved number of features-> 115
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:  5.8min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  6.4min finished
C:\Users\rajes\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best parameter on grid--> {'n_neighbors': 1, 'algorithm': 'auto'}
K NEAREST Neighbor: model:
Train Confusion Matrix:
[[226132      0]
 [     0 203518]]
Test Confusion Matrix:
[[46927  9627]
 [ 3895  1054]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train    1.00000  1.000000  1.000000       1.000000  1.000000
Test     0.09868  0.212972  0.134869       0.521373  0.780141


In [29]:
#Handling Imbalance problem
X_tr, X_test, y_tr, y_test =split_train_test(X,y)
'''
Use oversampling to handle imbalance problem
'''
text_file = open("cvres_knn_imb_grid.txt", "a+")
text_file.write('\nOVERSAMPLING:')
text_file.close()
# Class0 size is 10 times that of Class1 in train set
print('Y train before resampling',Counter(y_tr))
for i in range(15,100,15):
    print('Sampling strategy-->',i*0.01)
    write_file('Sampling strategy-->',i*0.01)      
    Imb_over = RandomOverSampler(random_state=42,sampling_strategy=i*0.01)
    X_train, y_train = Imb_over.fit_resample(X_tr, y_tr)
    print('Y train after resampling',Counter(y_train))
    write_file('Y train after resampling-->',Counter(y_train))       
    X_train=pd.DataFrame(X_train)
    y_train=pd.DataFrame(y_train)
    df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)
    run_grid_knn(df_new_train,y_train,df_new_test,y_test)


Y train before resampling Counter({0: 226132, 1: 19876})
Sampling strategy--> 0.15
Y train after resampling Counter({0: 226132, 1: 33919})


C:\Users\rajes\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Improved number of features-> 95
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:  3.0min remaining:   42.2s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  3.3min finished
C:\Users\rajes\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best parameter on grid--> {'n_neighbors': 1, 'algorithm': 'auto'}
K NEAREST Neighbor: model:
Train Confusion Matrix:
[[226132      0]
 [     0  33919]]
Test Confusion Matrix:
[[52026  4528]
 [ 4413   536]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000        1.00000  1.000000
Test    0.105845  0.108305  0.107061        0.51412  0.854625
Sampling strategy--> 0.3
Y train after resampling Counter({0: 226132, 1: 67839})


...

Best parameter on grid--> {'n_neighbors': 1, 'algorithm': 'auto'}
K NEAREST Neighbor: model:
Train Confusion Matrix:
[[226132      0]
 [     0 169599]]
Test Confusion Matrix:
[[52026  4528]
 [ 4413   536]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000        1.00000  1.000000
Test    0.105845  0.108305  0.107061        0.51412  0.854625
Sampling strategy--> 0.9
Y train after resampling Counter({0: 226132, 1: 203518})


C:\Users\rajes\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Improved number of features-> 86
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:  4.7min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  5.2min finished
C:\Users\rajes\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best parameter on grid--> {'n_neighbors': 1, 'algorithm': 'auto'}
K NEAREST Neighbor: model:
Train Confusion Matrix:
[[226132      0]
 [     0 203518]]
Test Confusion Matrix:
[[52026  4528]
 [ 4413   536]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000        1.00000  1.000000
Test    0.105845  0.108305  0.107061        0.51412  0.854625


In [ ]:
#Handling Imbalance problem
X_tr, X_test, y_tr, y_test =split_train_test(X,y)
'''
Use Undersampling to handle imbalance problem
'''
text_file = open("cvres_knn_imb_grid.txt", "a+")
text_file.write('\nUNDERSAMPLING:')
text_file.close()
# Class0 size is 10 times that of Class1 in train set
print('Y train before resampling',Counter(y_tr))
for i in range(15,100,15):
    print('Sampling strategy-->',i*0.01)
    write_file('Sampling strategy-->',i*0.01)
    Imb_under = RandomUnderSampler(random_state=42,sampling_strategy=i*0.01)
    X_train, y_train = Imb_under.fit_resample(X_tr, y_tr)
    print('Y train after resampling',Counter(y_train))
    write_file('Y train after resampling-->',Counter(y_train))         
    X_train=pd.DataFrame(X_train)
    y_train=pd.DataFrame(y_train)
    df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)
    run_grid_knn(df_new_train,y_train,df_new_test,y_test)
    

Y train before resampling Counter({0: 226132, 1: 19876})
Sampling strategy--> 0.15
Y train after resampling Counter({0: 132506, 1: 19876})


C:\Users\rajes\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Improved number of features-> 94
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:  1.5min remaining:   21.5s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.7min finished
C:\Users\rajes\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best parameter on grid--> {'n_neighbors': 1, 'algorithm': 'auto'}
K NEAREST Neighbor: model:
Train Confusion Matrix:
[[132506      0]
 [     0  19876]]
Test Confusion Matrix:
[[49366  7188]
 [ 4134   815]]
       PRECISION   RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.00000  1.000000        1.00000  1.000000
Test    0.101837  0.16468  0.125849        0.51879  0.815911
Sampling strategy--> 0.3
Y train after resampling Counter({0: 66253, 1: 19876})


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:   24.2s remaining:    5.6s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   26.7s finished
C:\Users\rajes\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best parameter on grid--> {'n_neighbors': 1, 'algorithm': 'auto'}
K NEAREST Neighbor: model:
Train Confusion Matrix:
[[33126     0]
 [    0 19876]]
Test Confusion Matrix:
[[35782 20772]
 [ 2852  2097]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000       1.000000  1.000000
Test    0.091696  0.423722  0.150766       0.528214  0.615889
Sampling strategy--> 0.75
Y train after resampling Counter({0: 26501, 1: 19876})


C:\Users\rajes\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Improved number of features-> 96
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
